In [7]:
china = {1: [('192.168.0.1', 3.805875778198242), ('192.168.0.1', 11.356830596923828), ('192.168.0.1', 7.946014404296875)], 5: [('200.89.165.86', 20.32613754272461), ('200.89.165.86', 44.97480392456055), ('200.89.165.86', 19.730091094970703)], 6: [('200.89.165.149', 22.628068923950195), ('200.89.165.149', 23.41485023498535), ('200.89.165.149', 25.152206420898438)], 7: [('200.89.165.5', 21.49200439453125), ('200.89.165.5', 12.93492317199707), ('200.89.165.5', 18.139123916625977)], 8: [('200.89.165.250', 21.111011505126953), ('200.89.165.250', 45.92084884643555), ('200.89.165.250', 23.07915687561035)], 9: [('185.70.203.32', 28.480052947998047), ('185.70.203.32', 15.650033950805664), ('185.70.203.32', 14.788150787353516)], 10: [('89.221.41.187', 192.2018527984619), ('89.221.41.187', 184.722900390625), ('89.221.41.187', 197.53623008728027)], 11: [('154.54.9.17', 205.6601047515869), ('154.54.9.17', 194.6709156036377), ('154.54.9.17', 181.37502670288086)], 12: [('154.54.47.29', 191.70904159545898), ('154.54.47.29', 185.87493896484375), ('154.54.47.29', 188.97700309753418)], 13: [('154.54.84.1', 189.4519329071045), ('154.54.84.1', 194.3809986114502), ('154.54.84.1', 196.1359977722168)], 14: [('154.54.30.162', 286.90314292907715), ('154.54.30.162', 303.71999740600586), ('154.54.30.162', 233.28399658203125)], 15: [('154.54.42.65', 277.446985244751), ('154.54.42.65', 269.7739601135254), ('154.54.42.65', 249.5870590209961)], 16: [('154.54.44.86', 298.88200759887695), ('154.54.44.86', 223.48904609680176), ('154.54.44.86', 245.73802947998047)], 17: [('154.54.42.102', 214.12110328674316), ('154.54.42.102', 301.6619682312012), ('154.54.42.102', 309.18407440185547)], 18: [('38.88.196.186', 320.6830024719238), ('38.88.196.186', 271.29292488098145), ('38.88.196.186', 256.6709518432617)], 19: [('101.4.117.169', 401.52716636657715), ('101.4.117.169', 381.27708435058594), ('101.4.117.169', 393.8729763031006)], 20: [('101.4.117.101', 397.51601219177246), ('101.4.117.101', 416.25308990478516), ('101.4.117.101', 436.51485443115234)], 21: [('101.4.115.254', 384.36007499694824), ('101.4.115.254', 378.52001190185547), ('101.4.115.254', 403.04088592529297)], 22: [('101.4.112.197', 387.8610134124756), ('101.4.112.197', 404.6061038970947), ('101.4.112.197', 395.7979679107666)], 23: [('202.112.38.10', 404.88409996032715), ('202.112.38.10', 395.00904083251953), ('202.112.38.10', 416.1510467529297)], 24: [('118.229.4.2', 422.6799011230469), ('118.229.4.2', 383.98098945617676), ('118.229.4.2', 396.2361812591553)], 25: [('118.229.2.66', 425.28486251831055), ('118.229.2.66', 378.49998474121094), ('118.229.2.66', 395.60914039611816)], 26: [('118.229.2.82', 383.9230537414551), ('118.229.2.82', 407.8798294067383), ('118.229.2.82', 412.0919704437256)], 28: [('166.111.4.100', 474.49207305908203), ('166.111.4.100', 470.64995765686035), ('166.111.4.100', 469.8450565338135)]}

In [72]:
import numpy as np
import pandas as pd
from scipy import stats
import math
import matplotlib.pyplot as plt

# The Modified Thompson tau 
def tau(n):
    # The critical students t value alpha = 0.05; df = n-2, https://stackoverflow.com/questions/19339305/python-function-to-get-the-t-statistic
    tinv = stats.t.ppf(1-0.025, n-2)  
    return tinv * (n - 1) / (math.sqrt(n) * math.sqrt(n - 2 + tinv ** 2))

#La onda es que vengan ordenados de menor a mayor, entonces si aparece un outlier, como dice en el paper volvemos a iterar pero sacando este ultimo
#Asi que el metodo devuelve la cantidad de no outliers 
def outliers_detection(hop_samples):
    n = len(hop_samples)
    mean = np.mean(hop_samples)
    std = np.std(hop_samples)
    t = tau(n)
    tS = t * std

    last_sample = hop_samples[-1]
    delta = np.absolute(last_sample - mean)

    if delta < tS:
        return n

    else:
        hop_samples.pop()
        return outliers_detection(hop_samples)


In [71]:
from scipy import stats
import math

# The Modified Thompson tau 
def tau(n):
    # The critical students t value alpha = 0.05; df = n-2, https://stackoverflow.com/questions/19339305/python-function-to-get-the-t-statistic
    tinv = stats.t.ppf(1-0.025, n-2)  
    return tinv * (n - 1) / (math.sqrt(n) * math.sqrt(n - 2 + tinv ** 2))
def outliers_detection(hop_samples):
    n = len(hop_samples)
    mean = np.mean(hop_samples)
    std = np.std(hop_samples)
    t = tau(n)
    tS = t * std

    last_sample = hop_samples[-1]
    delta = np.absolute(last_sample - mean)

    if delta < tS:
        return last_sample

    else:
        hop_samples.pop()
        return outliers_detection(hop_samples)

def mega(rs):
    df = pd.DataFrame()     
    last_rtt_inserted = 0
    for ttl in range(1, 30):
        if ttl not in rs: continue
        rtts = [ r[1] for r in rs[ttl] ]
        hosts = [ r[0] for r in rs[ttl] ]
        avg = np.average(rtts)
        df.set_value(ttl, 'avg', avg)
        df.set_value(ttl, 'min', np.min(rtts))
        df.set_value(ttl, 'std', np.std(rtts))
        df.set_value(ttl, 'host', max(hosts,key=hosts.count))
        df.set_value(ttl, 'ds', avg - last_rtt_inserted)
        last_rtt_inserted = avg

    return df
def mega2(rs, df):
    last_rtt_inserted = 0
    without_insert = 0
    for ttl in range(1, 30):
        without_insert = without_insert + 1
        if ttl not in rs: continue
        rtts = [ r[1] for r in rs[ttl] ]
        avg = np.average(rtts)
        if avg - last_rtt_inserted > 0:
            davg = (avg - last_rtt_inserted )/ without_insert
            for i in range(without_insert):
                df.set_value(ttl - i, 'ds2', davg)
            without_insert = 0
        last_rtt_inserted = avg
    
    return df
mega2(china, mega(china))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:34: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib/python3/dist-packages/ipykernel_launcher.py:35: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib/python3/dist-packages/ipykernel_launcher.py:36: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib/python3/dist-packages/ipykernel_launcher.py:37: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib/python3/dist-packages/ipykernel_launcher.py:38: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/usr/lib/python3/dist-packages/ipykernel_launcher.py:53: FutureWarning: set_value is depre

,avg,min,std,host,ds,ds2
1,7.702907,3.805876,3.087454,192.168.0.1,7.702907,7.702907
5,28.343678,19.730091,11.762500,200.89.165.86,20.640771,5.160193
6,23.731709,22.628069,1.054551,200.89.165.149,-4.611969,4.171663
7,17.522017,12.934923,3.520561,200.89.165.5,-6.209691,4.171663
8,30.037006,21.111012,11.260277,200.89.165.250,12.514989,4.171663
9,19.639413,14.788151,6.261171,185.70.203.32,-10.397593,85.923791
10,191.486994,184.722900,5.255386,89.221.41.187,171.847582,85.923791
11,193.902016,181.375027,9.929238,154.54.9.17,2.415021,2.415021
12,188.853661,185.874939,2.383359,154.54.47.29,-5.048354,2.234658
13,193.322976,189.451933,2.829457,154.54.84.1,4.469315,2.234658


In [66]:
range(10)[0]

0

In [57]:
pd.Series(ds).plt()

AttributeError: 'Series' object has no attribute 'plt'